##  MLOPS-Zoomcamp 
### cohort 2024

#### Module 1-intro

Prediction of a taxi ride duration

### Homework
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [2]:
# Imports
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

A - 16

B - 17

C - 18

D - 19

In [3]:
jan_df = pd.read_parquet('../data/raw/yellow_tripdata_2023-01.parquet')
fev_df = pd.read_parquet('../data/raw/yellow_tripdata_2023-02.parquet')



In [4]:
# I created this dict to iterate through the different datasets, because I hadn't looked into what I was supposed to do. This is not needed
df_dict = {}
df_dict = dict(zip(['January', 'February'], [jan_df, fev_df]))

In [5]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
fev_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [7]:
for month, df in df_dict.items():
    print(f"There are {len(df.columns)} columns in the {month} dataframe")

There are 19 columns in the January dataframe
There are 19 columns in the February dataframe


### Q1. Answer: 
**D** - 19 columns

### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

A - 32.59

B - 42.59

C - 52.59

D - 62.59

In [8]:
for month, df in df_dict.items():
    df_dict[month] = df.assign(duration=(df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']).apply(lambda td: td.total_seconds() / 60))

In [9]:
for month, df in df_dict.items():
    print(f"For the {month} dataset, the standard deviation of the trip duration was {df['duration'].std():.2f}\n", "*" * 10)

For the January dataset, the standard deviation of the trip duration was 42.59
 **********
For the February dataset, the standard deviation of the trip duration was 42.84
 **********


### Q2. Answer:
B - 42.59

### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

A - 90%

B - 92%

C - 95%

D - 98%

In [10]:
filtered_df_dict = {}
for month, df in df_dict.items():
    filtered_df = df[(df['duration']>1) & (df['duration']<=60)]
    print(f"For the {month} dataset, after removing outliers (>1 and <=60), the resulting dataset is {int((filtered_df.shape[0]/df.shape[0])*100)}% of the original one")
    filtered_df_dict[month] = filtered_df
        

For the January dataset, after removing outliers (>1 and <=60), the resulting dataset is 98% of the original one
For the February dataset, after removing outliers (>1 and <=60), the resulting dataset is 97% of the original one


### Q3. Answer:
D - 98%

### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

A - 2

B - 155

C - 345

D - 515

E - 715

In [11]:
# Convert the pickup and dropoff location IDs to strings and then to a list of dictionaries
train_df = filtered_df_dict['January'].copy()
train_df = train_df.astype({'PULocationID': 'str', 'DOLocationID': 'str'})


list_of_dicts = train_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')


In [12]:
# Initialize the DictVectorizer
vec = DictVectorizer()

# Fit and transform the data
feature_matrix = vec.fit_transform(list_of_dicts)

# Convert the feature matrix to a dense array for better readability (optional)
# feature_matrix_dense = feature_matrix.toarray()

feature_names = vec.get_feature_names_out()

# Output the results
# print("Feature Matrix (Dense):")
# print(feature_matrix_dense)
# print("\nFeature Names:")
# print(feature_names)
print("\n", "*"*12, "\n")
print(f"The dimensionality of the matrix is: {len(feature_names)}")
print("\n", "*"*12, "\n")


 ************ 

The dimensionality of the matrix is: 515

 ************ 



### Q4. Answer:
D - 515

### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

A - 3.64

B - 7.64

C - 11.64

D - 16.64

In [13]:
# Extract the target variable
target = train_df['duration'].values

# Train a plain linear regression model with default parameters
model = LinearRegression()
model.fit(feature_matrix, target)

# Predict on the training data
predictions = model.predict(feature_matrix)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(target, predictions))

print(f"RMSE on training data: {rmse:.2f}")

RMSE on training data: 7.65


### Q5. Answer:
B - 7.65

### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

A - 3.81

B - 7.81

C - 11.81

D - 16.81

In [14]:
validation_df = filtered_df_dict['February'].copy()
validation_df = validation_df.astype({'PULocationID': 'str', 'DOLocationID': 'str'})


validation_list_of_dicts = validation_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')


validation_feature_matrix = vec.transform(validation_list_of_dicts)


validation_predictions = model.predict(validation_feature_matrix)
validation_target = validation_df['duration'].values

validation_rmse = np.sqrt(mean_squared_error(validation_target, validation_predictions))

print(f"RMSE on validation data: {validation_rmse:.2f}")

RMSE on validation data: 7.81


### Q6. Answer:
B - 7.81